In [93]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [94]:
pip install matplotlib

## 1. Data Gathering

In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [2]:
train = pd.read_csv('train.csv')
calendar_df = pd.read_csv('calendar_preprocessed.csv')
prices_df = pd.read_csv('sell_prices_afcs2021.csv')
whole = pd.read_csv('whole.csv')

In [3]:
train

,id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,FOODS_3_001_TX_3_validation,0,2,1,3,0,0,1,0,2,...,0,0,0,0,2,0,0,5,2,0
1,FOODS_3_002_TX_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,FOODS_3_003_TX_3_validation,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,1,0,0
3,FOODS_3_004_TX_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
4,FOODS_3_005_TX_3_validation,0,1,1,0,0,1,2,0,0,...,0,4,2,1,1,0,1,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818,FOODS_3_823_TX_3_validation,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,2,0,2
819,FOODS_3_824_TX_3_validation,0,1,0,1,0,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
820,FOODS_3_825_TX_3_validation,0,0,0,2,1,1,0,0,1,...,1,1,1,3,1,0,0,4,2,0
821,FOODS_3_826_TX_3_validation,0,0,0,0,0,0,0,0,0,...,0,2,1,1,5,3,1,2,1,2


In [4]:
whole

,Unnamed: 0,Day,V2,V3,V4,V5,V6,V7,V8,V9,...,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,Event,Day_type
0,1,1,0,0,0,0,0,0,27,0,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,1
1,2,2,2,0,0,0,1,0,27,0,...,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,3,3,1,0,0,0,1,0,8,0,...,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,1
3,4,4,3,0,0,0,0,0,0,0,...,2,2011,d_4,NaN,NaN,NaN,NaN,1,0,1
4,5,5,0,0,0,0,0,0,0,0,...,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,1937,1937,0,0,2,0,2,0,0,14,...,6,2016,d_1965,NaN,NaN,NaN,NaN,0,0,1
1937,1938,1938,0,3,0,0,4,1,7,18,...,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,1
1938,1939,1939,0,1,0,0,2,2,6,29,...,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,1
1939,1940,1940,1,1,2,1,1,1,0,31,...,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,1


## 2. Data pre-processing

In [5]:
whole.rename(columns={"Event": "event_type_1_encoded"}, inplace = True)

In [6]:
le = LabelEncoder()
whole['event_name_1_encoded'] = le.fit_transform(whole['event_name_1'])
whole['event_type_2_encoded'] = le.fit_transform(whole['event_type_2'])
whole['event_name_2_encoded'] = le.fit_transform(whole['event_name_2'])

In [7]:
whole['day'] = pd.to_datetime(whole['Date'], format = '%Y/%m/%d').dt.day

In [8]:
whole.drop(["Unnamed: 0", "Date", "weekday", "d", "year", "event_name_1", "event_type_1",
            "event_type_1", "event_name_2", "event_type_2", "Day_type", "Day"], axis=1, inplace = True)

In [9]:
# Merge Products with their Price

product_list = prices_df['item_id'].to_list()

product_list_unique = []
[product_list_unique.append(x) for x in product_list if x not in product_list_unique]

def rename_products(dataset):
    
    index = 0
    for column in whole.columns:
        if column[0] == 'V':

            whole.rename(columns={column: product_list_unique[index]}, inplace = True)
            index += 1
            
rename_products(whole)

prices_df = prices_df.pivot(index = 'wm_yr_wk', columns = 'item_id', values = 'sell_price')
whole = pd.merge(whole, prices_df, on=['wm_yr_wk'])

whole.drop(["wm_yr_wk"], axis=1, inplace = True)

In [10]:
# Replace NaN values in the Price columns

def replace_nan_price(dataset):

    nan_values = dataset.isna()
    nan_columns = nan_values.any()

    columns_with_nan = dataset.columns[nan_columns].tolist()
    print("Number of columns with NaN values: ", len(columns_with_nan))

    for column in columns_with_nan:
        dataset[column] = dataset[column].interpolate().fillna(method='bfill')

replace_nan_price(whole)

Number of columns with NaN values:  512


In [11]:
# Add all external features to the end (so we first have time series and prices)

wday = whole.pop('wday')
month = whole.pop('month')
snap_CA = whole.pop('snap_CA')
event_type_1_encoded = whole.pop('event_type_1_encoded')
event_name_1_encoded = whole.pop('event_name_1_encoded')
event_type_2_encoded = whole.pop('event_type_2_encoded')
event_name_2_encoded = whole.pop('event_name_2_encoded')
day = whole.pop('day')
  
    
whole.insert(1645, 'wday', wday)
whole.insert(1646, 'month', month)
whole.insert(1647, 'snap_CA', snap_CA)
whole.insert(1648, 'event_type_1_encoded', event_type_1_encoded)
whole.insert(1649, 'event_name_1_encoded', event_name_1_encoded)
whole.insert(1650, 'event_type_2_encoded', event_type_2_encoded)
whole.insert(1651, 'event_name_2_encoded', event_name_2_encoded)
whole.insert(1652, 'day', day)

C:\Users\roxan\Anaconda3\envs\my-r-env\lib\site-packages\IPython\core\interactiveshell.py:3364: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


## 2.2 One-hot encoding and Standardization

In [12]:
# Make a copy so we have the initial data in case we need it later

dataset = whole.copy()

In [13]:
# Get one hot encodings for the qualitative data

dataset = pd.get_dummies(dataset, prefix=['wday_', 'month_', 'event_type_1_encoded_', 'event_name_1_encoded_',
                                          'event_type_2_encoded_', 'event_name_2_encoded_', 'day_'], 
                                  columns=['wday', 'month', 'event_type_1_encoded', 'event_name_1_encoded',
                                          'event_type_2_encoded', 'event_name_2_encoded', 'day'])

In [14]:
sc = MinMaxScaler(feature_range = (0, 1))
dataset.iloc[:, :823] = sc.fit_transform(dataset.iloc[:, :823])

sc_price = MinMaxScaler(feature_range = (0, 1))
dataset.iloc[:, 823:823*2] = sc_price.fit_transform(dataset.iloc[:, 823:823*2])

In [15]:
dataset_scaled = dataset.copy().to_numpy()

In [16]:
# Divide data into input X and labels y

def df_to_X_y(df, window_size, n_timeseries):
    
    X = []
    y = []
    
    for i in range(len(df) - window_size):

        X.append(df[i:i+window_size])
        y.append(df[i+window_size][:823])

    return np.array(X), np.array(y)

X, y = df_to_X_y(dataset_scaled, window_size = 28, n_timeseries = 823)

In [17]:
X_train, y_train = X[:1319], y[:1319]
X_val, y_val = X[1319:1885], y[1319:1885]
X_test, y_test = X[-28:], y[-28:]

In [18]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((1319, 28, 1741),
 (1319, 823),
 (566, 28, 1741),
 (566, 823),
 (28, 28, 1741),
 (28, 823))

## 3. Model

In [19]:
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.layers import LSTM
from keras.layers import Dropout
import tensorflow as tf
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [20]:
stacked_lstm = Sequential()

stacked_lstm.add(InputLayer((X_train.shape[1], X_train.shape[2])))

stacked_lstm.add(LSTM(512, return_sequences=True))
stacked_lstm.add(Dropout(0.2))
stacked_lstm.add(LSTM(512, return_sequences=True))
stacked_lstm.add(Dropout(0.2))
stacked_lstm.add(LSTM(256))
stacked_lstm.add(Dropout(0.2))

stacked_lstm.add(Dense(128, 'elu'))
stacked_lstm.add(Dropout(0.2))
stacked_lstm.add(Dense(y_train.shape[1], 'linear'))

stacked_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 512)           4616192   
                                                                 
 dropout (Dropout)           (None, 28, 512)           0         
                                                                 
 lstm_1 (LSTM)               (None, 28, 512)           2099200   
                                                                 
 dropout_1 (Dropout)         (None, 28, 512)           0         
                                                                 
 lstm_2 (LSTM)               (None, 256)               787456    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               3

In [21]:
stacked_lstm.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=[RootMeanSquaredError()])
cp = ModelCheckpoint('stacked_lstm/', save_best_only=True)

In [22]:
epoch = 200
batch_size = 16
history = stacked_lstm.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = epoch, batch_size = batch_size, callbacks=[cp])

Epoch 1/200
83/83 [==============================] - ETA: 0s - loss: 0.0149 - root_mean_squared_error: 0.1222

INFO:tensorflow:Assets written to: stacked_lstm\assets


INFO:tensorflow:Assets written to: stacked_lstm\assets


83/83 [==============================] - 45s 502ms/step - loss: 0.0149 - root_mean_squared_error: 0.1222 - val_loss: 0.0174 - val_root_mean_squared_error: 0.1318
Epoch 2/200
14/83 [====>.........................] - ETA: 25s - loss: 0.0120 - root_mean_squared_error: 0.1093

KeyboardInterrupt: 

In [37]:
stacked_lstm = load_model('stacked_lstm/') # now we have the best model in the memory

In [38]:
def plot_predictions(model, X, y):

    predictions = model.predict(X)

    return predictions

### Validation

In [39]:
val_results = plot_predictions(stacked_lstm, X_val, y_val)

C:\Users\roxan\Anaconda3\envs\my-r-env\lib\site-packages\ipykernel_launcher.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  del sys.path[0]


In [41]:
y_p_val = sc.inverse_transform(y_val)
predictions_val = sc.inverse_transform(val_results)

In [42]:
print("RMSE: ", np.sqrt(np.mean((predictions_val - y_p_val)**2)))

RMSE:  3.456956060735122


In [43]:
test_results = plot_predictions(stacked_lstm, X_test, y_test)
test_results

y_p = sc.inverse_transform(y_test)
predictions = sc.inverse_transform(test_results)

print("RMSE: ", np.sqrt(np.mean((predictions-y_p)**2)))

RMSE:  3.418272345934471


C:\Users\roxan\Anaconda3\envs\my-r-env\lib\site-packages\ipykernel_launcher.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  del sys.path[0]


### Test: Get final results only when the model is good enough

In [49]:
stacked_lstm.fit(X_val, y_val, epochs = 50, batch_size = batch_size)

Epoch 1/100
36/36 [==============================] - 15s 406ms/step - loss: 0.0109 - root_mean_squared_error: 0.1043
Epoch 2/100
36/36 [==============================] - 14s 386ms/step - loss: 0.0110 - root_mean_squared_error: 0.1047
Epoch 3/100
36/36 [==============================] - 14s 391ms/step - loss: 0.0108 - root_mean_squared_error: 0.1042
Epoch 4/100
36/36 [==============================] - 14s 387ms/step - loss: 0.0108 - root_mean_squared_error: 0.1038
Epoch 5/100
36/36 [==============================] - 14s 390ms/step - loss: 0.0108 - root_mean_squared_error: 0.1039
Epoch 6/100
36/36 [==============================] - 17s 478ms/step - loss: 0.0108 - root_mean_squared_error: 0.1041
Epoch 7/100
36/36 [==============================] - 18s 511ms/step - loss: 0.0108 - root_mean_squared_error: 0.1037
Epoch 8/100
36/36 [==============================] - 18s 496ms/step - loss: 0.0106 - root_mean_squared_error: 0.1032
Epoch 9/100
36/36 [==============================] - 17s 477ms/s

In [50]:
test_results_final = plot_predictions(stacked_lstm, X_test, y_test)
test_results_final

C:\Users\roxan\Anaconda3\envs\my-r-env\lib\site-packages\ipykernel_launcher.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  del sys.path[0]


array([[-0.00079461,  0.0459469 ,  0.05655087, ...,  0.28474656,
         0.13867368,  0.02837676],
       [ 0.09245439,  0.04343913,  0.13769671, ...,  0.10882202,
         0.17701617,  0.01268674],
       [ 0.03871036,  0.06232595,  0.09902569, ...,  0.1036424 ,
         0.13662216,  0.04454843],
       ...,
       [ 0.16022834,  0.01877886,  0.13106567, ...,  0.15347293,
         0.19599327,  0.04523205],
       [ 0.08183302,  0.03168511,  0.10376988, ...,  0.16454515,
         0.20761418,  0.02759552],
       [ 0.01203209, -0.01632668,  0.0474474 , ...,  0.07308254,
         0.17136136,  0.06260804]], dtype=float32)

In [51]:
y_p = sc.inverse_transform(y_test)
predictions_final = sc.inverse_transform(test_results_final)

In [52]:
print("RMSE: ", np.sqrt(np.mean((predictions_final-y_p)**2)))

RMSE:  3.0166817522104274


In [155]:
test_results_to_save = predictions_final.transpose()
test_results_to_save = pd.DataFrame(data=test_results_to_save)
test_results_to_save.to_csv('test_results_lstm5.csv')